<h1> Modeling Naive Bayes </h1>

<img src="https://ih1.redbubble.net/image.1450572675.6311/st,small,507x507-pad,600x600,f8f8f8.jpg" width="200" />

In [23]:
# from ipynb.fs.full.functions import *
from ipynb.fs.full.functions_stream_env import *
import spacy

In [24]:
# Data to create our model
dfa = pd.read_csv('../data/ama_comments.csv')
dfb = pd.read_csv('../data/ar_comments.csv')
df = pd.concat([dfa, dfb], axis=0)
df = CleanUp(df).df.copy()

In [25]:
# Model X, and y
#df = df.sample(n=df.shape[0], random_state=3)

X = df['body']
y = df['subreddit_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=3)

In [26]:
# TEST data (not part of train/test/split)
df1 = pd.read_csv('../data/2021-04-27_1812_AMA_comments.csv')
df2 = pd.read_csv('../data/2021-04-27_1812_AskReddit_comments.csv')
df_test_pred = pd.concat([df1, df2], axis=0)
df_test_pred = CleanUp(df_test_pred).df.copy()

In [27]:
df_test_pred = df_test_pred.sample(n=df_test_pred.shape[0], random_state=3)

X_new = df_test_pred['body']
y_new = df_test_pred['subreddit_binary'] 

In [28]:
### Did this Run???

In [29]:
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((7370,), (2457,), (7370,), (2457,))

<h2 style="color:red;"> (gridsearch) TfidfVectorizer(), LogisticRegression() </h2>
<h2 style="color:red;"> BEST! test score: 72.17% </h2>

#### BEST Params:
 - max_features: 5000
 - ngram_range: (1,2)
 - stop_words: None

In [30]:
# ### Best Train ###

gs_tv_lgr = ClassificationModel(make_pipeline(
    TfidfVectorizer(), 
    LogisticRegression()), 
    X_train, X_test, y_train, y_test,
    params={
        'tfidfvectorizer__ngram_range': [ (1, 1), (1, 2), (2, 2), (1, 3), (2, 3), (3, 3) ],
        'tfidfvectorizer__stop_words': [ 'english', None ],
        'tfidfvectorizer__max_features': [ 500, 1000, 2000, 5000 ]
}, verbose=3, mod_name='Train/Test Tfidf LogReg')


In [31]:
## FIT BEST onto X, and try on fresh data!!

X_gs_tv_lgr = ClassificationModel(make_pipeline(
    TfidfVectorizer(), 
    LogisticRegression()), 
    X_train=X, X_test=X_new, y_train=y, y_test=y_new,
    params={
        'tfidfvectorizer__ngram_range': [ (1, 1), (1, 2), (2, 2), (1, 3), (2, 3), (3, 3) ],
        'tfidfvectorizer__stop_words': [ 'english', None ],
        'tfidfvectorizer__max_features': [ 500, 1000, 2000, 5000 ]
}, verbose=3, mod_name='Xy/new Tfidf LogReg')


## Bayes Models

### First, our previous Bayesian models, without lemma-tization

<h2> (gridsearch) CountVectorizer(), MultinomialNB() </h2>


In [32]:
gs_cv_nb = ClassificationModel(make_pipeline(
    CountVectorizer(), 
    MultinomialNB()), 
    X_train, X_test, y_train, y_test,
    params={
        'countvectorizer__ngram_range': [ (1, 1), (1, 2), (2, 2), (1, 3), (2, 3), (3, 3) ],
        'countvectorizer__stop_words': [ 'english', None ],
        'countvectorizer__max_features': [ 500, 1000, 2000, 5000 ]
}, verbose=3, mod_name='Train/Test cVect nBayes')



In [33]:
X_gs_cv_nb = ClassificationModel(make_pipeline(
    CountVectorizer(), 
    MultinomialNB()), 
    X, X_new, y, y_new,
    params={
        'countvectorizer__ngram_range': [ (1, 1), (1, 2), (2, 2), (1, 3), (2, 3), (3, 3) ],
        'countvectorizer__stop_words': [ 'english', None ],
        'countvectorizer__max_features': [ 500, 1000, 2000, 5000 ]
}, verbose=3, mod_name='Xy/new cVect nBayes')



<h2> (gridsearch) TfidfVectorizer(), MultinomialNB() </h2>


In [34]:
gs_tv_nb = ClassificationModel(make_pipeline(
    TfidfVectorizer(), 
    MultinomialNB()), 
    X_train, X_test, y_train, y_test,
    params={
        'tfidfvectorizer__ngram_range': [ (1, 1), (1, 2), (2, 2), (1, 3), (2, 3), (3, 3) ],
        'tfidfvectorizer__stop_words': [ 'english', None ],
        'tfidfvectorizer__max_features': [ 500, 1000, 2000, 5000 ]
}, verbose=3, mod_name='Train/Test Tfidf nBayes')



In [35]:
X_gs_tv_nb = ClassificationModel(make_pipeline(
    TfidfVectorizer(), 
    MultinomialNB()), 
    X, X_new, y, y_new,
    params={
        'tfidfvectorizer__ngram_range': [ (1, 1), (1, 2), (2, 2), (1, 3), (2, 3), (3, 3) ],
        'tfidfvectorizer__stop_words': [ 'english', None ],
        'tfidfvectorizer__max_features': [ 500, 1000, 2000, 5000 ]
}, verbose=3, mod_name='Xy/new Tfidf nBayes')


In [36]:
compare_nb = pd.concat([gs_cv_nb.df, X_gs_cv_nb.df, gs_tv_nb.df, X_gs_tv_nb.df], axis=1)
compare_nb

,Train/Test cVect nBayes,Xy/new cVect nBayes,Train/Test Tfidf nBayes,Xy/new Tfidf nBayes
F1 Score,0.682448,0.607512,0.681319,0.644619
Recall Score,0.621138,0.571721,0.655285,0.629098
Accuracy,0.710623,0.632518,0.693122,0.654944
Balanced Accuracy,0.710732,0.632210,0.693168,0.654813
Precision Score,0.757185,0.648084,0.709507,0.660926
Average Precision Score,0.775027,0.660172,0.777313,0.690152
ROC AUC Score,0.778369,0.668874,0.781264,0.729180
True Positive,982.000000,683.000000,897.000000,671.000000
False Negative,245.000000,303.000000,330.000000,315.000000
False Positive,466.000000,418.000000,424.000000,362.000000


<h3> STOP </h3>
<h2> LEMMA-TIME! </h2>
<img src="https://i.imgflip.com/xv6aq.jpg" />
<strong><em> (Just a reminder that if I want to use Spacy, I need to change my conda env) </em></strong>

<h2> (gridsearch) CountVectorizer(<em>tokenizer=lemmatized</em>), MultinomialNB() </h2>

#### BEST Params:
 - max_features: 5000
 - ngram_range: (1,1)
 - stop_words: None

In [37]:
nlp = spacy.load("en_core_web_sm")

In [38]:
def lemmatized(sentence):
    doc = nlp(sentence)
    return [token.lemma_ for token in doc]

In [39]:

cvect = CountVectorizer(tokenizer=lemmatized)
X_train_cv_lemma = cvect.fit_transform(X_train)
X_test_cv_lemma = cvect.transform(X_test)

X_cv_lemma = cvect.fit_transform(X)
X_new_cv_lemma = cvect.transform(X_new)


In [40]:

tvect = TfidfVectorizer(tokenizer=lemmatized)
X_train_tv_lemma = tvect.fit_transform(X_train)
X_test_tv_lemma = tvect.transform(X_test)

X_tv_lemma = tvect.fit_transform(X)
X_new_tv_lemma = tvect.transform(X_new)


In [41]:
gs_cv_nb = GridSearchCV(make_pipeline(CountVectorizer(tokenizer=lemmatized), MultinomialNB()), {})
gs_cv_nb.fit(X_train, y_train)
y_gs_cv_nb = gs_cv_nb.predict(X_test)
print_classification_stats(y_test, y_gs_cv_nb)

{'F1 Score': 0.6839907192575406,
 'Recall Score': 0.5991869918699188,
 'Accuracy Score': 0.7228327228327228,
 'Balanced Accuracy': 0.7229838789830441,
 'Precision Score': 0.7967567567567567,
 'True Positive': 737,
 'False Negative': 493,
 'False Positive': 188,
 'True Negative': 1039,
 'Calculated Accuracy': 0.7228327228327228,
 'Calculated Precision': 0.7967567567567567,
 'Calculated Sensitivity': 0.5991869918699188,
 'Calculated Specificity': 0.8467807660961695}

In [42]:
X_gs_cv_nb = GridSearchCV(make_pipeline(CountVectorizer(tokenizer=lemmatized), MultinomialNB()), {})
X_gs_cv_nb.fit(X, y)
y_X_gs_cv_nb = X_gs_cv_nb.predict(X_new)
print_classification_stats(y_new, y_X_gs_cv_nb)


{'F1 Score': 0.6083001705514497,
 'Recall Score': 0.548155737704918,
 'Accuracy Score': 0.6488277268093782,
 'Balanced Accuracy': 0.6483172197652378,
 'Precision Score': 0.6832694763729247,
 'True Positive': 535,
 'False Negative': 441,
 'False Positive': 248,
 'True Negative': 738,
 'Calculated Accuracy': 0.6488277268093782,
 'Calculated Precision': 0.6832694763729247,
 'Calculated Sensitivity': 0.548155737704918,
 'Calculated Specificity': 0.7484787018255578}

#### Try it without Gridsearch
...and it is the same - which is good! But the values are not better than Logistic Regression.

In [43]:
bayes_cv_lemma = MultinomialNB()
bayes_cv_lemma.fit(X_train_cv_lemma, y_train)

bayes_cv_lemma.predict(X_test_cv_lemma)
# (X_test_cv_lemma.shape, y_test.shape)

bayes_cv_lemma_dict = print_classification_stats(y_test, bayes_cv_lemma.predict(X_test_cv_lemma))
bayes_cv_lemma_dict

{'F1 Score': 0.6839907192575406,
 'Recall Score': 0.5991869918699188,
 'Accuracy Score': 0.7228327228327228,
 'Balanced Accuracy': 0.7229838789830441,
 'Precision Score': 0.7967567567567567,
 'True Positive': 737,
 'False Negative': 493,
 'False Positive': 188,
 'True Negative': 1039,
 'Calculated Accuracy': 0.7228327228327228,
 'Calculated Precision': 0.7967567567567567,
 'Calculated Sensitivity': 0.5991869918699188,
 'Calculated Specificity': 0.8467807660961695}

In [44]:
bayes_cv_lemma_df = pd.DataFrame([bayes_cv_lemma_dict]).T.rename(columns={0: 'Train/Test cVect nBayes no GS'}, errors="raise")
bayes_cv_lemma_df

,Train/Test cVect nBayes no GS
F1 Score,0.683991
Recall Score,0.599187
Accuracy Score,0.722833
Balanced Accuracy,0.722984
Precision Score,0.796757
True Positive,737.000000
False Negative,493.000000
False Positive,188.000000
True Negative,1039.000000
Calculated Accuracy,0.722833


In [45]:
X_bayes_cv_lemma = MultinomialNB()
X_bayes_cv_lemma.fit(X_cv_lemma, y)
X_bayes_cv_lemma.predict(X_new_cv_lemma)

X_bayes_cv_lemma_dict = print_classification_stats(y_new, X_bayes_cv_lemma.predict(X_new_cv_lemma))
X_bayes_cv_lemma_dict

{'F1 Score': 0.6083001705514497,
 'Recall Score': 0.548155737704918,
 'Accuracy Score': 0.6488277268093782,
 'Balanced Accuracy': 0.6483172197652378,
 'Precision Score': 0.6832694763729247,
 'True Positive': 535,
 'False Negative': 441,
 'False Positive': 248,
 'True Negative': 738,
 'Calculated Accuracy': 0.6488277268093782,
 'Calculated Precision': 0.6832694763729247,
 'Calculated Sensitivity': 0.548155737704918,
 'Calculated Specificity': 0.7484787018255578}

In [46]:
y_new_bayes_cv_lemma_df = pd.DataFrame([X_bayes_cv_lemma_dict]).T.rename(columns={0: 'Xy/new cVect nBayes no GS'}, errors="raise")
y_new_bayes_cv_lemma_df

,Xy/new cVect nBayes no GS
F1 Score,0.608300
Recall Score,0.548156
Accuracy Score,0.648828
Balanced Accuracy,0.648317
Precision Score,0.683269
True Positive,535.000000
False Negative,441.000000
False Positive,248.000000
True Negative,738.000000
Calculated Accuracy,0.648828


<h2> (gridsearch) TfidfVectorizer(<em>tokenizer=lemmatized</em>), MultinomialNB() </h2>

#### BEST Params:
 - max_features: 5000
 - ngram_range: (1,1)
 - stop_words: None

In [47]:
gs_tv_nb = GridSearchCV(make_pipeline(TfidfVectorizer(tokenizer=lemmatized), MultinomialNB()), {})
gs_tv_nb.fit(X_train, y_train)
y_gs_tv_nb = gs_tv_nb.predict(X_test)
print_classification_stats(y_test, y_gs_tv_nb)


{'F1 Score': 0.6611889801836637,
 'Recall Score': 0.5560975609756098,
 'Accuracy Score': 0.7146927146927147,
 'Balanced Accuracy': 0.7148865962987259,
 'Precision Score': 0.8152562574493445,
 'True Positive': 684,
 'False Negative': 546,
 'False Positive': 155,
 'True Negative': 1072,
 'Calculated Accuracy': 0.7146927146927147,
 'Calculated Precision': 0.8152562574493445,
 'Calculated Sensitivity': 0.5560975609756098,
 'Calculated Specificity': 0.8736756316218419}

In [48]:
X_gs_tv_nb = GridSearchCV(make_pipeline(TfidfVectorizer(tokenizer=lemmatized), MultinomialNB()), {})
X_gs_tv_nb.fit(X, y)
y_X_gs_tv_nb = X_gs_tv_nb.predict(X_new)
print_classification_stats(y_new, y_X_gs_tv_nb)


{'F1 Score': 0.5861062463514303,
 'Recall Score': 0.514344262295082,
 'Accuracy Score': 0.6386340468909276,
 'Balanced Accuracy': 0.6380037741495694,
 'Precision Score': 0.6811397557666214,
 'True Positive': 502,
 'False Negative': 474,
 'False Positive': 235,
 'True Negative': 751,
 'Calculated Accuracy': 0.6386340468909276,
 'Calculated Precision': 0.6811397557666214,
 'Calculated Sensitivity': 0.514344262295082,
 'Calculated Specificity': 0.7616632860040567}

#### Try it without Gridsearch
...and it is the same - which is good! But the values are not better than Logistic Regression.

In [49]:
bayes_tv_lemma = MultinomialNB()
bayes_tv_lemma.fit(X_train_tv_lemma, y_train)

bayes_tv_lemma.predict(X_test_tv_lemma)
# (X_test_cv_lemma.shape, y_test.shape)

bayes_tv_lemma_dict = print_classification_stats(y_test, bayes_tv_lemma.predict(X_test_tv_lemma))
bayes_tv_lemma_dict

{'F1 Score': 0.6611889801836637,
 'Recall Score': 0.5560975609756098,
 'Accuracy Score': 0.7146927146927147,
 'Balanced Accuracy': 0.7148865962987259,
 'Precision Score': 0.8152562574493445,
 'True Positive': 684,
 'False Negative': 546,
 'False Positive': 155,
 'True Negative': 1072,
 'Calculated Accuracy': 0.7146927146927147,
 'Calculated Precision': 0.8152562574493445,
 'Calculated Sensitivity': 0.5560975609756098,
 'Calculated Specificity': 0.8736756316218419}

In [50]:
bayes_tv_lemma_df = pd.DataFrame([bayes_tv_lemma_dict]).T.rename(columns={0: 'Train/Test tVect nBayes no GS'}, errors="raise")
bayes_tv_lemma_df

,Train/Test tVect nBayes no GS
F1 Score,0.661189
Recall Score,0.556098
Accuracy Score,0.714693
Balanced Accuracy,0.714887
Precision Score,0.815256
True Positive,684.000000
False Negative,546.000000
False Positive,155.000000
True Negative,1072.000000
Calculated Accuracy,0.714693


In [51]:
X_bayes_tv_lemma = MultinomialNB()
X_bayes_tv_lemma.fit(X_tv_lemma, y)
X_bayes_tv_lemma.predict(X_new_tv_lemma)

X_bayes_tv_lemma_dict = print_classification_stats(y_new, X_bayes_tv_lemma.predict(X_new_tv_lemma))
X_bayes_tv_lemma_dict

{'F1 Score': 0.5861062463514303,
 'Recall Score': 0.514344262295082,
 'Accuracy Score': 0.6386340468909276,
 'Balanced Accuracy': 0.6380037741495694,
 'Precision Score': 0.6811397557666214,
 'True Positive': 502,
 'False Negative': 474,
 'False Positive': 235,
 'True Negative': 751,
 'Calculated Accuracy': 0.6386340468909276,
 'Calculated Precision': 0.6811397557666214,
 'Calculated Sensitivity': 0.514344262295082,
 'Calculated Specificity': 0.7616632860040567}

In [52]:
y_new_bayes_tv_lemma_df = pd.DataFrame([X_bayes_tv_lemma_dict]).T.rename(columns={0: 'Xy/new tVect nBayes no GS'}, errors="raise")
y_new_bayes_tv_lemma_df

,Xy/new cVect nBayes no GS
F1 Score,0.586106
Recall Score,0.514344
Accuracy Score,0.638634
Balanced Accuracy,0.638004
Precision Score,0.681140
True Positive,502.000000
False Negative,474.000000
False Positive,235.000000
True Negative,751.000000
Calculated Accuracy,0.638634


# COMPARE

In [53]:
compare_nb = pd.concat([compare_nb, bayes_cv_lemma_df, y_new_bayes_cv_lemma_df, bayes_tv_lemma_df, y_new_bayes_tv_lemma_df], axis=1)
compare_nb

,Train/Test cVect nBayes,Xy/new cVect nBayes,Train/Test Tfidf nBayes,Xy/new Tfidf nBayes,Train/Test cVect nBayes no GS,Xy/new cVect nBayes no GS,Train/Test tVect nBayes no GS,Xy/new cVect nBayes no GS
F1 Score,0.682448,0.607512,0.681319,0.644619,0.683991,0.608300,0.661189,0.586106
Recall Score,0.621138,0.571721,0.655285,0.629098,0.599187,0.548156,0.556098,0.514344
Accuracy,0.710623,0.632518,0.693122,0.654944,NaN,NaN,NaN,NaN
Balanced Accuracy,0.710732,0.632210,0.693168,0.654813,0.722984,0.648317,0.714887,0.638004
Precision Score,0.757185,0.648084,0.709507,0.660926,0.796757,0.683269,0.815256,0.681140
Average Precision Score,0.775027,0.660172,0.777313,0.690152,NaN,NaN,NaN,NaN
ROC AUC Score,0.778369,0.668874,0.781264,0.729180,NaN,NaN,NaN,NaN
True Positive,982.000000,683.000000,897.000000,671.000000,737.000000,535.000000,684.000000,502.000000
False Negative,245.000000,303.000000,330.000000,315.000000,493.000000,441.000000,546.000000,474.000000
False Positive,466.000000,418.000000,424.000000,362.000000,188.000000,248.000000,155.000000,235.000000


#### The Gridsearch with Tfidf Vectorization and without Lemma-tization yeilded better results.